In [3]:
import gzip
from collections import defaultdict
import random

def readGz(f):
    for l in gzip.open(f):
        yield eval(l)

### Rating baseline: compute averages for each user, or return the global average if we've never seen the user before

allRatings = []
userRatings = defaultdict(list)
for l in readGz("train.json.gz"):
    user,business = l['reviewerID'],l['itemID']
    allRatings.append(l['rating'])
    userRatings[user].append(l['rating'])

globalAverage = sum(allRatings) / len(allRatings)
userAverage = {}
for u in userRatings:
    userAverage[u] = sum(userRatings[u]) / len(userRatings[u])

predictions = open("predictions_Rating.txt", 'w')
for l in open("pairs_Rating.txt"):
    if l.startswith("reviewerID"):
    #header
        predictions.write(l)
        continue
    u,i = l.strip().split('-')
    if u in userAverage:
        predictions.write(u + '-' + i + ',' + str(userAverage[u]) + '\n')
    else:
        predictions.write(u + '-' + i + ',' + str(globalAverage) + '\n')

predictions.close()



In [4]:
### Would-purchase baseline: just rank which businesses are popular and which are not, and return '1' if a business is among the top-ranked
businessCount = defaultdict(int)
totalPurchases = 0
purchases = defaultdict(set)
purchasesTest = defaultdict(set)
items = set()
count = 0
allData = defaultdict(int)
allPurchases = 0
categoryTrain = defaultdict(set)
itemCategories = defaultdict(str)
for l in readGz("train.json.gz"):
    user,business = l['reviewerID'],l['itemID']
    allData[business] += 1
    allPurchases += 1
    if count <= 100000:
        businessCount[business] += 1
        totalPurchases += 1
        count += 1
    else:
        purchasesTest[l['reviewerID']].add(l['itemID'])
    purchases[l['reviewerID']].add(l['itemID'])
    categoryTrain[l['reviewerID']].add(l['categoryID'])
    itemCategories[l['itemID']] = l['categoryID']
    items.add(l['itemID'])

#Non-purchase pairs
negatives = defaultdict(set)
itemsList = list(items)
users = list(purchases.keys())
count = 0
while count < 100000:
    cus = random.choice(users)
    item = random.choice(itemsList)
    if item not in purchases[cus]:
        negatives[cus].add(item)
        count += 1
mostPopular = [(businessCount[x], x) for x in businessCount]
mostPopular.sort()
mostPopular.reverse()

mostPopAll = [(allData[x], x) for x in allData]
mostPopAll.sort()
mostPopAll.reverse()

validation = []
for u in negatives.keys():
    for i in negatives[u]:
        validation.append((u,i,0))
for u in purchasesTest.keys():
    for i in purchasesTest[u]:
        validation.append((u,i,1))
            
factor = 0.48
maxAcc = 0
maxFactor = 0
for k in range(1000):
    factor += 0.0001
    return1 = set()
    count = 0
    for ic, i in mostPopular:
        count += ic
        return1.add(i)
        if count > (totalPurchases*factor): break
    correct = 0
    for p in validation:
        if (p[1] in return1):
            if (p[2] == 1):
                correct += 1
        elif (p[2] == 0):
            correct += 1
    if ((correct/len(validation)) > maxAcc):
        maxAcc = (correct/len(validation))
        maxFactor = factor
print('Validation accuracy is: {} at factor {}'.format(maxAcc, maxFactor))
factor = 0.5
count = 0
for ic, i in mostPopular:
        count += ic
        return1.add(i)
        if count > (totalPurchases*factor): break
correct = 0
for p in validation:
    if (p[1] in return1):
        if (p[2] == 1):
            correct += 1
    elif (p[2] == 0):
        correct += 1
print('Validation accuracy is: {} at factor {}'.format(correct/len(validation), 0.5))
popular = 1
#Predicting with the popularity model
if (popular == 1):
    return1 = set()
    count = 0
    for ic, i in mostPopular:
        count += ic
        return1.add(i)
        if count > (totalPurchases*maxFactor): break

    predictions = open("predictions_Purchase.txt", 'w')
    for l in open("pairs_Purchase.txt"):
        if l.startswith("reviewerID"):
        #header
            predictions.write(l)
            continue
        u,i = l.strip().split('-')
        if i in return1:
            predictions.write(u + '-' + i + ",1\n")
        else:
            predictions.write(u + '-' + i + ",0\n")
    predictions.close()
else:
    #Predicting with the category model
    predictions = open("predictions_Purchase.txt", 'w')
    for l in open("pairs_Purchase.txt"):
        if l.startswith("reviewerID"):
        #header
            predictions.write(l)
            continue
        u,i = l.strip().split('-')
        if itemCategories[i] in categoryTrain[u]:
            predictions.write(u + '-' + i + ",1\n")
        else:
            predictions.write(u + '-' + i + ",0\n")
    predictions.close()


Validation accuracy is: 0.629718891566747 at factor 0.5151999999999961
Validation accuracy is: 0.6290188705661169 at factor 0.5


In [ ]:
### Category prediction baseline: Just consider some of the most common words from each category

catDict = {
  "Women": 0,
  "Men": 1,
  "Girls": 2,
  "Boys": 3,
  "Baby": 4
}

categoryCountsTrain = defaultdict(int)
userCategoryCountsTrain = defaultdict(defaultdict(int))
categoryCountsVal = defaultdict(int)
userCategoryCountsVal = defaultdict(defaultdict(int))
count = 0
for l in readGz("train.json.gz"):
    if count < 100000:
        categoryCountsTrain[l['categoryID']] += 1
        userCategoryCountsTrain[l['reviewerID']][l['categoryID']] += 1
        count += 1
    else:
        categoryCountsVal[l['categoryID']] += 1
        userCategoryCountsVal[l['reviewerID']][l['categoryID']] += 1
    user,business = l['reviewerID'],l['itemID']
    allRatings.append(l['rating'])
    userRatings[user].append(l['rating'])

    


predictions = open("predictions_Category.txt", 'w')
predictions.write("reviewerID-reviewHash,category\n")
for l in readGz("test_Category.json.gz"):
    cat = catDict['Women'] # If there's no evidence, just choose the most common category in the dataset
    words = l['reviewText'].lower()
    if 'wife' in words:
        cat = catDict['Women']
    if 'husband' in words:
        cat = catDict['Men']
    if 'daughter' in words:
        cat = catDict['Girls']
    if 'son' in words:
        cat = catDict['Boys']
    if 'baby' in words:
        cat = catDict['Baby']
    predictions.write(l['reviewerID'] + '-' + l['reviewHash'] + "," + str(cat) + "\n")

predictions.close()

In [ ]:
import operator
translator = str.maketrans('', '', string.punctuation)
wordCount = defaultdict(int)
categoryWordCount = defaultdict(dict)
for l in readGz("test_Category.json.gz"):
    sentence = l['reviewText'].translate(translator).lower()
    words = sentence.split()
    for word in words:
        wordCount[word] += 1
        if 'categoryID' in l.keys():
            categoryWordCount[l['categoryID']][word] += 1
topWords = sorted(wordCount.items(), key=operator.itemgetter(1))
topWords.reverse()
topWords = topWords[:500]


total = sum(pair[1] for pair in topWords)
frequencies = [(pair[0],pair[1]/total) for pair in topWords]